# tutorial

- https://docs.sqlalchemy.org/en/20/tutorial
- https://hellowac.github.io/technology/python/sqlalchemy/

## relationship and foreignkey

* https://blog.51cto.com/u_15301988/3125714
* https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html

# in-memory


In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [13]:
import contextlib
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import relationship, Mapped
from datetime import datetime
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    DateTime,
    String,
    func,
    UniqueConstraint
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.session import Session


session_make: sessionmaker = sessionmaker(bind=engine)
# session: Session = session_make()

engine = create_engine("sqlite:///sales.db", echo=True)


Base = declarative_base()


class Customers(Base):
    __tablename__ = "customers"
    id = Column(Integer, primary_key=True)

    name = Column(String)
    address = Column(String)
    email = Column(String)

class BaseMixin:
    """model的基类,所有model都必须继承"""
    id = Column(Integer, primary_key=True)
    created_at = Column(DateTime, nullable=False, default=datetime.now)
    updated_at = Column(DateTime, nullable=False, default=datetime.now, onupdate=datetime.now, index=True)
    deleted_at = Column(DateTime)  # 可以为空, 如果非空, 则为软删


@contextlib.contextmanager
def get_session():
    s: Session = session_make()
    try:
        yield s
        s.commit()
    except Exception as e:
        s.rollback()
        raise e
    finally:
        s.close()


class User(Base, BaseMixin):
    __tablename__ = "user"

    Name = Column(String(36), nullable=False)
    Phone = Column(String(36), nullable=False, unique=True)

Base.metadata.create_all(engine)  # type: ignore

2023-11-04 23:34:36,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-04 23:34:36,676 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2023-11-04 23:34:36,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-04 23:34:36,678 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2023-11-04 23:34:36,678 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-04 23:34:36,679 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2023-11-04 23:34:36,680 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-04 23:34:36,681 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	created_at DATETIME NOT NULL, 
	updated_at DATETIME NOT NULL, 
	deleted_at DATETIME, 
	"Name" VARCHAR(36) NOT NULL, 
	"Phone" VARCHAR(36) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE ("Phone")
)


2023-11-04 23:34:36,682 INFO sqlalchemy.engine.Engine [no key 0.00059s] ()
2023-11-04 23:34:36,861 INFO sqlalchemy.engine.Engine CREATE INDEX ix_user_updated_at ON user (updated_

In [14]:
with get_session() as s:
    print(s.query(User).first())

2023-11-04 23:35:23,194 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-04 23:35:23,196 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.created_at AS user_created_at, user.updated_at AS user_updated_at, user.deleted_at AS user_deleted_at, user."Name" AS "user_Name", user."Phone" AS "user_Phone" 
FROM user
 LIMIT ? OFFSET ?
2023-11-04 23:35:23,196 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (1, 0)
None
2023-11-04 23:35:23,198 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
type(Session)

sqlalchemy.orm.session.sessionmaker

In [7]:
result: list[Customers] = session.query(Customers).all()

for row in result:
    print("Name: ", row.name, "Address:", row.address, "Email:", row.email)

2023-11-04 22:12:23,610 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers
2023-11-04 22:12:23,611 INFO sqlalchemy.engine.Engine [cached since 62.57s ago] ()
Name:  Ravi Kumar Address: Station Road Nanded Email: ravi@gmail.com


In [8]:
session.query(Customers).count()

2023-11-04 22:14:06,776 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers) AS anon_1
2023-11-04 22:14:06,777 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ()


1

In [9]:
result = session.query(Customers).filter(Customers.id>2, Customers.name.like('Ra%'))
for row in result:
   print ("ID:", row.id, "Name: ",row.name, "Address:",row.address, "Email:",row.email)

2023-11-04 22:30:03,269 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.address AS customers_address, customers.email AS customers_email 
FROM customers 
WHERE customers.id > ? AND customers.name LIKE ?
2023-11-04 22:30:03,271 INFO sqlalchemy.engine.Engine [generated in 0.00080s] (2, 'Ra%')
